# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [5]:
alerce = alerce_tns()

# Load recent candidates

# CHANGE DATE

In [28]:
refstring = "20201115b"
candidates_dir = "candidates"
candidates = open("%s/%s.csv" % (candidates_dir, refstring), encoding='utf-8-sig').read().splitlines()
print(candidates, len(candidates))

['ZTF20acqnfkh'] 1


# Select host galaxies using aladin widget

In [29]:
alerce.select_hosts(candidates, catsHTM=False, vizier=False)

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.6/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '8:24:47.8419 -20:48:47.1802': No astronomical object found :
  (error.line, error.msg))


In [30]:
alerce.candidate_host_names

{'ZTF20acqnfkh': 'WISEA J082447.77-204847.1'}

In [31]:
alerce.candidate_host_redshifts

{'ZTF20acqnfkh': 'nan'}

# Manual modifications

In [32]:
#oid = "ZTF20acpkmtf"
#oid = 'ZTF20aclzygy'#ZTF20acgigfo'
#oid = "ZTF20aaqrmxg" #"ZTF20aakdppm" #ZTF19acxrihd"
#oid = 'ZTF20acoqrpm'
#alerce.candidate_host_names[oid] = "M31"
#alerce.candidate_host_redshifts[oid] = "nan" #"0.041" #nan"

# Clean nans or -999 or -9999

In [33]:
for k in list(alerce.candidate_host_redshifts.keys()):
    if alerce.candidate_host_redshifts[k] in ["nan", "-99", "-999.0", "-9999.0"]:
        print(k)
        alerce.candidate_host_redshifts.pop(k)

ZTF20acqnfkh


# Save hosts

In [34]:
df_host_names = pd.DataFrame.from_dict(alerce.candidate_host_names, orient='index', columns=["host_id"])
df_host_names.index.name='oid'
df_host_redshifts = pd.DataFrame.from_dict(alerce.candidate_host_redshifts, orient='index', columns=["z"])
df_host_redshifts.index.name='oid'
df_hosts = df_host_names.join(df_host_redshifts)
df_hosts.to_csv("candidates/%s_hosts.csv" % refstring)
df_hosts.head(100)

,host_id,z
oid,,
ZTF20acqnfkh,WISEA J082447.77-204847.1,NaN


# Display host galaxy properties

In [35]:
for oid in candidates:
    if oid in alerce.candidate_host_redshifts.keys():
        print(oid, alerce.candidate_host_names[oid], alerce.candidate_host_redshifts[oid])
    elif oid in alerce.candidate_host_names.keys():
        print(oid, alerce.candidate_host_names[oid])
    else:
        alerce.candidate_host_names[oid] = ""
        alerce.candidate_host_redshifts[oid] = ""
        print(oid)

ZTF20acqnfkh WISEA J082447.77-204847.1


# Sandbox or real

In [36]:
sandbox = False # True: test in sandbox, False: send to real website

# Get API key and url

In [37]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api="https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api="https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


# Prepare TNS report

** CHECK AUTHOR LIST!!! **

(key is used to query only at this step)

In [38]:
len(candidates)

1

In [39]:
reporter = "F. Forster, F.E. Bauer, A. Munoz-Arancibia, L. Hernandez-Garcia, G. Pignata, L. Galbany, G. Pignata, E. Camacho, J. Silva-Farfan, A. Mourao, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
report = {"at_report": {}}
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    #if oid in ["ZTF20aauqmwf"]:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid in ["ZTF20acqnfkh ", "ZTF20acneabh"]:#ZTF20acplctd":#aamtgd": #ZTF19acymtgr": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    #if oid == "ZTF20ablnjwh": #ZTF20abkmxrm":#ZTF20abgfgne":
    #    continue
    #if oid in ["ZTF20acmevcc", "ZTF20acmctng", "ZTF20acmdvzk"]: #== "ZTF20acgknep":#"ZTF20acdcdqq": #ZTF20acaehey": #ZTF20abyjoxb":#"ZTF20abqjddh":#ZTF20abkxqzv":#ZTF20abgbfyk":
    #    continue
    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        report["at_report"]["%i" % counter] = sn_report
        counter += 1



 0 ZTF20acqnfkh
http://alerce.online/object/ZTF20acqnfkh


SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20acqnfkh 


# Manual modifications

In [40]:
#report['at_report']['0']['remarks'] = "Fast rising transient (g-rise > 1.48 mag/day)"#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (g-rise > 0.16+-0.01 mag/day, r-rise > 0.24+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxrihd '

In [41]:
# note that the space at the end is necessary
#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (r-rise > 0.43+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20acgigfo '

# Display report

In [42]:
print(json.dumps(report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.09",
                "units": "arcsec",
                "value": "-20.8131056"
            },
            "discovery_data_source_id": "48",
            "discovery_datetime": "2020-11-14 11:31:29.004",
            "host_name": "WISEA J082447.77-204847.1",
            "internal_name": "ZTF20acqnfkh",
            "non_detection": {
                "archival_remarks": "ZTF non-detection limits not available",
                "archiveid": "0"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
                        "exptime": "30",
                        "filter_value": "111",
                        "flux": "18.9069576263428",
                        "flux_error": "0.0956765487790108",
                        "flux_units": "1",
                        "instrumen

# Save report

In [43]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(report, outfile)

# Send to TNS

**WARNING: key is used to send candidates!**

In [44]:
json_report

'candidates/20201115b.json'

In [45]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

# Show output

In [46]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":54947,"recieved_data":{"at_report":{"0":{"ra":{"value":126.19934135,"error":0.09,"units":"arcsec"},"dec":{"value":-20.8131056,"error":0.09,"units":"arcsec"},"reporting_group_id":74,"discovery_data_source_id":48,"reporter":"F. Forster, F.E. Bauer, A. Munoz-Arancibia, L. Hernandez-Garcia, G. Pignata, L. Galbany, G. Pignata, E. Camacho, J. Silva-Farfan, A. Mourao, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2020-11-14 11:31:29.004","at_type":1,"internal_name":"ZTF20acqnfkh","remarks":"SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF20acqnfkh ","non_detection":{"archiveid":0,"archival_remarks":"ZTF non-detection limits not a

# Check the following link and confirm that the latest SNe are from the reported day

https://wis-tns.weizmann.ac.il/search?&discovered_period_value=1&discovered_period_units=days&reporting_groupid[]=74&discovery_data_source_id=48&num_page=50&format=html&edit[type]=&edit[objname]=&edit[id]=&display[redshift]=1&display[hostname]=1&display[host_redshift]=1&display[source_group_name]=1&display[classifying_source_group_name]=1&display[discovering_instrument_name]=0&display[classifing_instrument_name]=0&display[programs_name]=0&display[internal_name]=1&display[isTNS_AT]=0&display[public]=1&display[end_pop_period]=0&display[spectra_count]=1&display[discoverymag]=1&display[discmagfilter]=1&display[discoverydate]=1&display[discoverer]=1&display[remarks]=0&display[sources]=0&display[bibcode]=0&display[ext_catalogs]=0&sort=desc&order=discoverydate